# Review NLP Helpfulness Prediction PretrainedModels (Toys)

## Creating Spark Session & Importing All Necessary Libraries

In [1]:
import findspark
findspark.init()

In [2]:
# Import Spark NLP
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql import functions as F

import sparknlp

# Start SparkSession with Spark NLP
# start() functions has 5 parameters: gpu, spark23, spark24, spark32, and memory
# sparknlp.start(gpu=True) will start the session with GPU support
# sparknlp.start(spark23=True) is when you have Apache Spark 2.3.x installed
# sparknlp.start(spark24=True) is when you have Apache Spark 2.4.x installed
# sparknlp.start(spark32=True) is when you have Apache Spark 3.2.x installed
# sparknlp.start(memory="16G") to change the default driver memory in SparkSession
spark = sparknlp.start(gpu = True, spark32= True)

### Reading in Data

In [3]:
df = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .csv(r"C:\Users\kenne\OneDrive\Desktop\UChicago\Python\Final Project Big Data Amazon Review\Amazon Review Kaggle Data\amazon_reviews_us_Toys_v1_00.tsv",inferSchema=True,header=True, sep='\t' )

## Code Cleaning

In [4]:
df.show(1, vertical = True, truncate = False)

-RECORD 0---------------------------------------
 marketplace       | US                         
 customer_id       | 18778586                   
 review_id         | RDIJS7QYB6XNR              
 product_id        | B00EDBY7X8                 
 product_parent    | 122952789                  
 product_title     | Monopoly Junior Board Game 
 product_category  | Toys                       
 star_rating       | 5                          
 helpful_votes     | 0                          
 total_votes       | 0                          
 vine              | N                          
 verified_purchase | Y                          
 review_headline   | Five Stars                 
 review_body       | Excellent!!!               
 review_date       | 2015-08-31                 
only showing top 1 row



In [5]:
df = df.select('product_id', 'star_rating', 'product_category', 'review_headline', 'review_body', 'helpful_votes', 'total_votes')

In [6]:
df.show(1, vertical = True, truncate = False)

-RECORD 0------------------------
 product_id       | B00EDBY7X8   
 star_rating      | 5            
 product_category | Toys         
 review_headline  | Five Stars   
 review_body      | Excellent!!! 
 helpful_votes    | 0            
 total_votes      | 0            
only showing top 1 row



In [7]:
df.dropna().count()

4863165

In [8]:
from pyspark.sql.functions import col,isnan, when, count
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+----------+-----------+----------------+---------------+-----------+-------------+-----------+
|product_id|star_rating|product_category|review_headline|review_body|helpful_votes|total_votes|
+----------+-----------+----------------+---------------+-----------+-------------+-----------+
|         0|          8|               8|            355|        740|            8|          8|
+----------+-----------+----------------+---------------+-----------+-------------+-----------+



In [9]:
df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
df2.count()
df2.show(5, vertical = True)

-RECORD 0----------------
 product_id       | 1    
 star_rating      | 8    
 product_category | 8    
 review_headline  | 536  
 review_body      | 6301 
 helpful_votes    | 8    
 total_votes      | 8    



In [10]:
df = df.filter(col('total_votes') > 10)
df.count()

155991

In [11]:
df = df.fillna("", "review_body")
df = df.fillna("", "review_headline")

In [12]:
df = df.withColumn('review_text', F.concat('review_headline', F.lit(" "), 'review_body'))
df.show(1, vertical = True, truncate = False)

-RECORD 0-----------------------------------------------------------------------------------------------------------
 product_id       | B00LE9MZMW                                                                                      
 star_rating      | 5                                                                                               
 product_category | Toys                                                                                            
 review_headline  | Five Stars                                                                                      
 review_body      | Great cube, moves and twists easily. It may seem costly, but it's worth every penny.            
 helpful_votes    | 23                                                                                              
 total_votes      | 27                                                                                              
 review_text      | Five Stars Great cube, moves and twists easi

In [13]:
df.filter((col("review_text").isNull()) | ( col("review_text") == "")).show(1, vertical = True, truncate = False)

(0 rows)



In [14]:
df.select([count(when((col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns]).show(5, vertical = True)

-RECORD 0---------------
 product_id       | 0   
 star_rating      | 0   
 product_category | 0   
 review_headline  | 6   
 review_body      | 55  
 helpful_votes    | 0   
 total_votes      | 0   
 review_text      | 0   



In [15]:
df = df.withColumn('helpful_ratio', F.col('helpful_votes') / F.col('total_votes'))
df.show(1, vertical = True, truncate = False)

-RECORD 0-----------------------------------------------------------------------------------------------------------
 product_id       | B00LE9MZMW                                                                                      
 star_rating      | 5                                                                                               
 product_category | Toys                                                                                            
 review_headline  | Five Stars                                                                                      
 review_body      | Great cube, moves and twists easily. It may seem costly, but it's worth every penny.            
 helpful_votes    | 23                                                                                              
 total_votes      | 27                                                                                              
 review_text      | Five Stars Great cube, moves and twists easi

In [16]:
df.filter(col('helpful_ratio') < 0).count()

0

In [17]:
df = df.withColumn('helpful', when(col("helpful_ratio") < 0.5, 0).otherwise(1))

In [18]:
df.show(5, vertical = True)

-RECORD 0--------------------------------
 product_id       | B00LE9MZMW           
 star_rating      | 5                    
 product_category | Toys                 
 review_headline  | Five Stars           
 review_body      | Great cube, moves... 
 helpful_votes    | 23                   
 total_votes      | 27                   
 review_text      | Five Stars Great ... 
 helpful_ratio    | 0.8518518518518519   
 helpful          | 1                    
-RECORD 1--------------------------------
 product_id       | B00LOV24VC           
 star_rating      | 1                    
 product_category | Toys                 
 review_headline  | If you want a qua... 
 review_body      | i bought this cha... 
 helpful_votes    | 21                   
 total_votes      | 21                   
 review_text      | If you want a qua... 
 helpful_ratio    | 1.0                  
 helpful          | 1                    
-RECORD 2--------------------------------
 product_id       | B00U1KA12G    

In [19]:
df = df.sample(False, 0.5, seed = 0)

## USE Pipeline

In [20]:
#USE (Universal Sentence Encoder) Sentence Embedding
document = DocumentAssembler()\
    .setInputCol("review_text")\
    .setOutputCol("document")
    
embeddingsSentence = UniversalSentenceEncoder.load('tfhub_use_en_2.4.0_2.4_1587136330099') \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

classifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("prediction")\
  .setLabelColumn("helpful")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

use_clf_pipeline = Pipeline(
    stages = [
        document,
        embeddingsSentence,
        classifierdl
    ])

In [21]:
%%time
pipeline2 = use_clf_pipeline.fit(df)

Wall time: 52.5 s


In [22]:
%%time
final = pipeline2.transform(df)

Wall time: 50.5 ms


In [23]:
metrics = final.select('helpful','product_id',"prediction.result")

## USE Results

In [24]:
%%time
from sklearn.metrics import classification_report, accuracy_score
metrics_final = metrics.toPandas()
metrics_final['result'] = metrics_final['result'].apply(lambda x: x[0])
metrics_final['result'] = metrics_final['result'].astype('int')

print(classification_report(metrics_final.helpful, metrics_final.result))
print(accuracy_score(metrics_final.helpful, metrics_final.result))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      8984
           1       0.89      1.00      0.94     69173

    accuracy                           0.89     78157
   macro avg       0.44      0.50      0.47     78157
weighted avg       0.78      0.89      0.83     78157

0.8850518827488261
Wall time: 17.7 s


C:\Users\kenne\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## BERT_uncased Embedding Pipeline

In [25]:
#BERT Sentence Embedding
document = DocumentAssembler()\
    .setInputCol("review_text")\
    .setOutputCol("document")
    
bert_cmlm = BertSentenceEmbeddings.load('sent_bert_base_uncased_en_2.6.0_2.4_1598346203624')\
.setInputCols(["document"])\
.setOutputCol("sentence_embeddings")

classifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("prediction")\
  .setLabelColumn("helpful")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

use_clf_pipeline = Pipeline(
    stages = [
        document,
        embeddingsSentence,
        classifierdl
    ])

In [26]:
%%time
pipeline2 = use_clf_pipeline.fit(df)

Wall time: 53.1 s


In [27]:
%%time
final = pipeline2.transform(df)

Wall time: 39 ms


In [28]:
metrics = final.select('helpful','product_id',"prediction.result")

## BERT_uncased Embedding Results

In [29]:
%%time
from sklearn.metrics import classification_report, accuracy_score
metrics_final = metrics.toPandas()
metrics_final['result'] = metrics_final['result'].apply(lambda x: x[0])
metrics_final['result'] = metrics_final['result'].astype('int')

print(classification_report(metrics_final.helpful, metrics_final.result))
print(accuracy_score(metrics_final.helpful, metrics_final.result))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      8984
           1       0.89      1.00      0.94     69173

    accuracy                           0.89     78157
   macro avg       0.44      0.50      0.47     78157
weighted avg       0.78      0.89      0.83     78157

0.8850518827488261
Wall time: 17.2 s
